In [1]:
import pandas as pd
import re
import utils.functions as f
import nltk
from utils.stop_words_custom import my_stop_words as stop_words
pd.set_option('display.max_colwidth', -1)

In [2]:
df = f.get_data("classified_auto/all.xlsx")
df = df.dropna(subset=['description'])

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21278 entries, 0 to 21277
Data columns (total 2 columns):
description    21278 non-null object
class          21278 non-null object
dtypes: object(2)
memory usage: 498.7+ KB


In [4]:
df["class"].value_counts()

negative    11007
positive    10271
Name: class, dtype: int64

In [5]:
df.head(3)

,description,class
0,É chegada a hora do Congresso reagir com um pedido de impeachment de Bolsonaro e sua equipe. Estão colocando o país e a população em risco graças a incompetência administrativa para resolver os problemas internos porque no exterior nossa diplomacia não existe #ForaBolsonaroEBando,negative
1,"4 meses de letargia do governo Bolsonaro, a equipe econômica é um fracasso, os índices apontam aumento da inflação, desemprego e recessão e esses tontos trocaram a tradição diplomática pacífica por um tom bélico demagogo. Nossas instituições estão falidas #ForaBolsonaroEBando",negative
2,"em 2 horas de entrevista, LULA apresentou mais propostas que o Bolsonaro em 30 anos de política.\n\nem 2 horas, Lula desmascarou a farsa da Lava Jato e da Previdência.\n\nem apenas 2 horas, #LulaInspira o povo ir à luta contra o desmonte do país.\n\nPOR ISSO MORREM DE MEDO QUE ELE FALE",negative


In [6]:
df_class_factor = pd.factorize(df["class"])
df_class_factor

(array([0, 0, 0, ..., 0, 1, 1]),
 Index(['negative', 'positive'], dtype='object'))

In [7]:
df["class_f"] = df_class_factor[0]
df.tail(3)

,description,class,class_f
21275,@senadorhumberto @jairbolsonaro Bolsonaro e um verme e todos os seus nazistas 😉\n\n#BolsonaroTeEnganouBabaca,negative,0
21276,"Vou aproveitar minha embriaguez pra falar uma coisa: Bolsonaro, vai tomar no cu! #EuAcreditoNoBolsonaro",positive,1
21277,"#EuAcreditoNoBolsonaro (11) e todo este discurso, tem um só objetivo: dizer que eu não acredito no bolsonaro! Não, eu acredito em Deus. E que, assim como ele o levantou da morte, ele o dará sabedoria e força para governar.\nDeus não o salvou a toa. E se Deus está do lado dele,",positive,1


In [8]:
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
count_vect = CountVectorizer()

In [10]:
X_train_counts = count_vect.fit_transform(df["description"])

In [11]:
X_train_counts.shape

(21278, 30231)

In [12]:
from sklearn.feature_extraction.text import TfidfTransformer

In [13]:
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)

In [14]:
X_train_tf = tf_transformer.transform(X_train_counts)

In [15]:
X_train_tf.shape

(21278, 30231)

In [16]:
tfidf_transformer = TfidfTransformer()

In [17]:
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [18]:
X_train_tfidf.shape

(21278, 30231)

In [19]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, df["class_f"])

In [20]:
df_predict = f.get_data('data/07_repeated_descriptions_deleted/without_repeated_descriptions.xlsx')

In [21]:
df_predict = df_predict.dropna(subset=['description'])

In [22]:
#df_predict_description = df_predict["description"].apply(f.removeSpecialCharactere)

In [23]:
docs_new_predict = df_predict["description"]
X_new_counts_predict = count_vect.transform(docs_new_predict)
X_new_tfidf_predict = tfidf_transformer.transform(X_new_counts_predict)

In [24]:
predicted = clf.predict(X_new_tfidf_predict)

In [25]:
positive = 0
negative = 0
neutral = 0
positive_description = []
negative_description = []
neutral_description = []

for doc, category in zip(docs_new_predict, predicted):
    if category == 1:
        category = "positive"
        positive = positive + 1
        positive_description.append(doc)
#     elif category == 1:
#         category = "neutral"
#         neutral = neutral + 1
#         neutral_description.append(doc)
    else:
        category = "negative"
        negative = negative + 1
        negative_description.append(doc)
#     print('%r => %s' % (doc, category))

In [26]:
positive

122702

In [27]:
negative

104070

In [28]:
for tweet in negative_description[0:100]:
    print(tweet)
    print("----------------------------------")

Que surpresa, um lobista da pesca industrial, filho de armadores e aliado do Bolsonaro, puxa, não esperava mesmo https://t.co/Dw51K6vEzl
----------------------------------
Justiça nega pedido de Flávio Bolsonaro para interromper investigação sobre Queiroz https://t.co/O2Urarfi0e
----------------------------------
Bolsonaro vai a banco e na volta encontra protesto na porta de seu condomínio https://t.co/rY0YacbNzO
----------------------------------
@UOLNoticias Por falar nisso cadê o Queiroz,Bolsonaro nem toca no assunto.
----------------------------------
Filosofia e Sociologia no Governo Bolsonaro! Pq investir menos nesses cursos? https://t.co/VFgylnzecg
----------------------------------
Se eu encontrasse um gênio da lâmpada e pudesse fazer 3 pedidos, um deles seria um vídeo do bolsonaro dando o cu.
----------------------------------
@blogdojosias @UOLNoticias @UOL Bolsonaro é o verdadeiro Papagaio de Pirata,acha que é o dono do Mundo.
----------------------------------
@peaarchno Ex

In [29]:
for tweet in positive_description[0:100]:
    print(tweet)
    print("----------------------------------")

Maia diz que cabe ao Congresso, e não a Bolsonaro, autorizar guerra https://t.co/K2i07XWdGd
----------------------------------
@PAULOSP001 @NaitzSprite @caiopilan @Tosilva17 @RaphaelSG14 @Ideias_Radicais @pauloap Isso se você nao tiver vivendo numa Venezuela, queria ver tu falar isso se Haddad fosse presidente com desarmamento e regulação dos meios de comunicação como estava no plano de governo dele... É facil ser liberal surfando na onda conservadora. Agradeça ao Bolsonaro. :)
----------------------------------
@RodrigoMaia Mandar nossos homens para Venezuela? Esse negócio só deve ser aprovado se for um exército formado pela família Bolsonaro e seus militantes, fora isso duvido que nossas famílias apóiem essa intervenção ridícula. Não dão conta nem do próprio país e querer se meter no dos outros?
----------------------------------
@apcify eu toda vez que vejo o bolsonaro:
----------------------------------
@brasil247 Esse folhetim esquerdista não fala de outro nome que não seja Bolson

In [ ]:
import spacy
import pandas as pd
import random
import re
import spacy
import time
import utils.functions as f
from utils.stop_words_custom import my_stop_words as stop_words
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
nlp = spacy.load('pt_core_news_sm')
pd.set_option('display.max_colwidth', -1)

In [ ]:
def clean_tweet(tweet):
    cleaned = f.toFixPunctuation(tweet)
    cleaned = f.changePatterns(cleaned)
    #cleaned = f.removeHashtag(cleaned)
    cleaned = f.removeLink(cleaned)
    #cleaned = f.removeMention(cleaned)
    cleaned = f.remove_punctuation(cleaned)
    cleaned = f.toFixBlankSpaces(cleaned)
    cleaned = f.remove_spaces_between_words(cleaned)
    return cleaned

In [ ]:
df_positive = f.get_data('data/10_classify_tweets/classify_tweets.xlsx')
df_negative = f.get_data('data/11_classified/negative.xlsx')
df_positive2 = f.get_data('data/11_classified/positive.xlsx')

In [ ]:
df_negative.shape

In [ ]:
df_positive.shape

In [ ]:
df_positive2.shape

In [ ]:
df_positive.columns

In [ ]:
df_positive = df_positive[df_positive["class"]=="positive"]

In [ ]:
df_positive.head(1)

In [ ]:
df_negative.head(1)

In [ ]:
df_negative.columns = ["description"]
df_negative.head(1)

In [ ]:
df_negative["class"] = "negative"

In [ ]:
df_negative.head(1)

In [ ]:
df_negative["class"].value_counts()

In [ ]:
df_positive2.columns = ["description"]
df_positive2["class"] = "positive"

In [ ]:
df_all_classified = pd.concat([df_negative, df_positive, df_positive2])
df_all_classified["class"].value_counts()

In [ ]:
df_description_cleaned = df_all_classified['description'].apply(clean_tweet)
train_X, test_X, train_y, test_y = train_test_split(df_description_cleaned, df_all_classified['class'], test_size=0.5, random_state=42, stratify=df_all_classified['class'])

In [ ]:
# vectorizer = CountVectorizer(ngram_range=(1,2))
vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 2))

In [ ]:
train_X = vectorizer.fit_transform(train_X)
test_X = vectorizer.transform(test_X)

clf = MultinomialNB()
clf.fit(train_X, train_y)

print("The accuracy on the test set is %.2f. The ngram representation had %i features." % (clf.score(test_X, test_y), train_X.shape[1]))

In [ ]:
#print(vectorizer.get_feature_names())

In [ ]:
# prediction = clf.predict(vectorizer.transform(["O Bolsonaro vai se reeleger em 2022."]))
# print(prediction)

In [ ]:
# prediction = clf.predict(vectorizer.transform(["O Bolsonaro matou a Marielle."]))
# print(prediction)

In [ ]:
# bow_matrix = vectorizer.fit_transform(df["description"])
# print(bow_matrix.shape)

In [ ]:
# bow_matrix = vectorizer.fit_transform(description_series)
# print(bow_matrix.shape)

In [ ]:
df_all = f.get_data('data/07_repeated_descriptions_deleted/without_repeated_descriptions.xlsx')

In [ ]:
df_all.info()

In [ ]:
df_all.drop(['guid','link','title','user_id','profile_image_url','lang','code','coords','source','user_statuses_count','user_friends_count','user_followers_count','user_location'], axis=1, inplace=True)

In [ ]:
df_all.info()

In [ ]:
df_all.dropna(subset=['description'],inplace=True)

In [ ]:
df_all.info()

In [ ]:
def preditor(x):
    prediction = clf.predict(vectorizer.transform([clean_tweet(x)]))
    return prediction[0]

In [ ]:
start_time = time.time()
y = df_all["description"].apply(preditor)
df_all["predicted"] = y
print(time.time() - start_time)

In [ ]:
df_all["predicted"].value_counts()

In [ ]:
df_all[df_all["predicted"]=="positive"].iloc[0:100]

In [ ]:
df_all[df_all["predicted"]=="negative"].iloc[0:100]

In [ ]:
df_positive = df_all[df_all["predicted"]=="positive"]
df_positive.info()

In [ ]:
df_negative = df_all[df_all["predicted"]=="negative"]
df_negative.info()

In [ ]:
# amostra_positivos = []
# for x in range(50):
#      amostra_positivos.append(random.randint(0,len(df_positive)))

In [ ]:
# amostra_positivos

In [ ]:
# amostra_negativos = []
# for x in range(50):
#      amostra_negativos.append(random.randint(0,len(df_negative)))

In [ ]:
# amostra_negativos

In [ ]:
amostra_all = []
for x in range(100):
     amostra_all.append(random.randint(0,len(df_all)))

In [ ]:
print(amostra_all)

In [ ]:
df_all.iloc[amostra_all][0:10]["description"]

In [ ]:
df_all.iloc[amostra_all][10:20]["description"]

In [ ]:
df_all.iloc[amostra_all][20:30]["description"]

In [ ]:
df_all.iloc[amostra_all][30:40]["description"]

In [ ]:
df_positive.iloc[amostra_positivos][40:50]["description"]

In [ ]:
df_all.iloc[amostra_all][50:60]["description"]

In [ ]:
df_all.iloc[amostra_all][60:70]["description"]

In [ ]:
df_all.iloc[amostra_all][70:80]["description"]

In [ ]:
df_all.iloc[amostra_all][80:90]["description"]

In [ ]:
df_all.iloc[amostra_all][90:100]["description"]

In [ ]:
writer = pd.ExcelWriter(r'data/12_positive.xlsx', engine='xlsxwriter',options={'strings_to_urls': False})
df_positive.to_excel(writer)
writer.close()

In [ ]:
writer = pd.ExcelWriter(r'data/12_negative.xlsx', engine='xlsxwriter',options={'strings_to_urls': False})
df_negative.to_excel(writer)
writer.close()